<h1 style="font-size:3rem;color:blue;"> Correlated Microstructures Elements Descriptor (CMED)</h1>

<img src="CMEDprocess.png">

# Librarys

In [ ]:
import cv2
import numpy as np
import math

# CMED Main Function

## Get_CMED function

- **img**. Image to describe
- **qc1**. Color cuantification number in H (8)
- **qc2**. Color cuantification number in S (3)
- **qc3**. Color cuantification number in V (3)
- **Qo**. Orientation cuantification number (6)
- **Qi**. Intensity cuantification number (10)
- **Par**. Paramters vector
  - **te**. Estructure type ()
  - **ne**. Number of structures ()
  - **pe**. Position of structure detection (DC or CC)

In [ ]:
def Get_CMED (img, qc1, qc2, qc3, Qo, Qi, Par):
    [Row, Col]=img.shape[:2]
    [te, ne, pe]=Par[:3]
    Qc = qc1*qc2*qc3
    #-----------------------------GET HSV IMAGE
    img_HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    HSV = NorImage(img_HSV,179,255,255)    #OpenCV gets H [0 - 179] S [0-255] V [0-255]
    #-----------------COLOR QUANTIZATION IN HSV COLOR SPACE
    ImageC=GetColorMap(HSV,qc1,qc2,qc3,Row,Col)
    #MULTI-DIMENSIONAL TEXTURE ORIENTATION DETECTION IN HSV COLOR SPACE (MD-TOD)
    hsv = Cylin2Cart(HSV,Row,Col)
    ImageO=GetOriMap(hsv,Qo,Row,Col)
    #-------------------------INTENSITY MAP EXTRACTION
    ImageI=GetIntMap(HSV,Qi,Row,Col)
    #----------------------STRUCTURES DETECTION & CORRELATION
    if pe=='DC':
        #---------------CORRELATED MICROSTRUCTURE IDENTIFICATION
        mmC=Get_MicroMap(ImageO,ImageI,ImageC,Row,Col)# MICRO-COLOR MAP
        mmO=Get_MicroMap(ImageI,ImageC,ImageO,Row,Col)# MICRO-ORIENTATION MAP
        mmI=Get_MicroMap(ImageC,ImageO,ImageI,Row,Col)# MICRO-INTENSITY MAP
        #---------------------STRUCTURE ELEMENTS DETECTION
        hEC=Get_SEH(ImageC,Row,Col,te,ne)
        hEO=Get_SEH(ImageO,Row,Col,te,ne)
        hEI=Get_SEH(ImageI,Row,Col,te,ne)
    elif pe=='CC':
        [hEC,mmC]=Get_MicroMapStr(ImageO,ImageI,ImageC,Row,Col,te,ne)# MICRO-COLOR MAP FEATURES
        [hEO,mmO]=Get_MicroMapStr(ImageI,ImageC,ImageO,Row,Col,te,ne)# MICRO-ORIENTATION MAP FEATURES
        [hEI,mmI]=Get_MicroMapStr(ImageC,ImageO,ImageI,Row,Col,te,ne)# MICRO-INTENSITY MAP
    #----------------------------HISTOGRAM REPRECENTATION
    hC= Get_MSR(mmC, Qc, Row, Col) # micro-structure representation Color
    hO= Get_MSR(mmO, Qo, Row, Col) # micro-structure representation Orientation
    hI= Get_MSR(mmI, Qi, Row, Col) # micro-structure representation Intensity
    #---------------------------------FINALL VECTOR
    CMED=np.concatenate((hEC,hEO,hEI,hC,hO,hI))
    return CMED

## *NorImage* Function

In [ ]:
def NorImage (img,c1,c2,c3):
    Nimg=np.array([img[:,:,0]/c1,img[:,:,1]/c2,img[:,:,2]/c3])
    return Nimg

## *GetColorMap* Function

In [ ]:
def GetColorMap(HSV,qc1,qc2,qc3,Row,Col):
    R=Row-2
    C=Col-2
    CM=np.zeros((R,C))
    HI=0
    SI=0
    VI=0
    for i in range (R):
        for j in range (C):
            HI=round(HSV[0, i+1, j+1] * qc1)
            if HI >= qc1:
                HI=HI-1
            SI=round(HSV[1, i+1, j+1] * qc2)
            if SI >= qc2:
                SI=SI-1
            VI=round(HSV[2, i+1, j+1] * qc3)
            if VI >= qc3:
                VI=VI-1
            CM[i,j] = qc3 * qc2 * HI + qc3 * SI + VI
    return CM

## *GetOriMap* Function

In [ ]:
def GetOriMap(HSV,Qo,Row,Col):
    hsv=Cylin2Cart(HSV,Row,Col)
    #MD-TOD
    ori=GetMD_TOD(hsv,Row,Col)
    OM = ori*(Qo-1)
    return OM

### *Cylin2Cart* Function

In [ ]:
def Cylin2Cart(HSV,Row,Col):
    M=np.zeros((3,Row,Col))
    G=HSV[0,:,:]*360
    R=G*np.pi/180
    M[0,:,:] = HSV[1,:,:] * np.cos(R)
    M[1,:,:] = HSV[1,:,:] * np.sin(R)
    M[2,:,:] = HSV[2,:,:]
    return M

### *GetMD_TOD* Function

In [ ]:
def GetMD_TOD(img,Row,Col):
    R=Row-2
    C=Col-2
    Txy=np.zeros((R,C))
    Td=np.zeros((R,C))
    #Sobel directions
    for i in range (R):
        for j in range (C):
            # Horizontal
            rh = (img[0,i,j+2] + 2*img[0,i+1,j+2] + img[0,i+2,j+2]) - (img[0,i,j] + 2*img[0,i+1,j] + img[0,i+2,j])
            gh = (img[1,i,j+2] + 2*img[1,i+1,j+2] + img[1,i+2,j+2]) - (img[1,i,j] + 2*img[1,i+1,j] + img[1,i+2,j])
            bh = (img[2,i,j+2] + 2*img[2,i+1,j+2] + img[2,i+2,j+2]) - (img[2,i,j] + 2*img[2,i+1,j] + img[2,i+2,j])
            # Vertical
            rv = (img[0,i+2,j] + 2*img[0,i+2,j+1] + img[0,i+2,j+2]) - (img[0,i,j] + 2*img[0,i,j+1] + img[0,i,j+2])
            gv = (img[1,i+2,j] + 2*img[1,i+2,j+1] + img[1,i+2,j+2]) - (img[1,i,j] + 2*img[1,i,j+1] + img[1,i,j+2])
            bv = (img[2,i+2,j] + 2*img[2,i+2,j+1] + img[2,i+2,j+2]) - (img[2,i,j] + 2*img[2,i,j+1] + img[2,i,j+2])
            # Obtain angle by the product of the vectors  H*V= |H|*|V|*cos(angle) ∴ angle = arcos (H*V/|H|*|V|)
            Txy[i,j] = GetAnglePV(rh,gh,bh,rv,gv,bv)
            # +45 degree
            rpd = (img[0,i,j+1] + 2*img[0,i,j+2] + img[0,i+1,j+2]) - (img[0,i+1,j] + 2*img[0,i+2,j] + img[0,i+2,j+1])
            gpd = (img[1,i,j+1] + 2*img[1,i,j+2] + img[1,i+1,j+2]) - (img[1,i+1,j] + 2*img[1,i+2,j] + img[1,i+2,j+1])
            bpd = (img[2,i,j+1] + 2*img[2,i,j+2] + img[2,i+1,j+2]) - (img[2,i+1,j] + 2*img[2,i+2,j] + img[2,i+2,j+1])
            # -45 degree
            rnd = (img[0,i+2,j+1] + 2*img[0,i+2,j+2] + img[0,i+1,j+2]) - (img[0,i+1,j] + 2*img[0,i,j] + img[0,i,j+1])
            gnd = (img[1,i+2,j+1] + 2*img[1,i+2,j+2] + img[1,i+1,j+2]) - (img[1,i+1,j] + 2*img[1,i,j] + img[1,i,j+1])
            bnd = (img[2,i+2,j+1] + 2*img[2,i+2,j+2] + img[2,i+1,j+2]) - (img[2,i+1,j] + 2*img[2,i,j] + img[2,i,j+1])
            # Obtain angle by the product of the vectors  d45*d135= |d45|*|d135|*cos(angle) ∴ angle = arcos (d45*d135/|d45|*|d135|)
            Td[i,j] = GetAnglePV(rpd,gpd,bpd,rnd,gnd,bnd)
    T=(Txy+Td)/2
    return T

#### *GetAnglePV* Function

In [ ]:
def GetAnglePV(r1,g1,b1,r2,g2,b2):
    gr1 = math.sqrt(pow(r1,2) + pow(g1,2) + pow(b1,2)) # |H| OR |d45|
    gr2 = math.sqrt(pow(r2,2) + pow(g2,2) + pow(b2,2)) # |V| OR |d135|
    gr3 = r1*r2 + g1*g2 + b1*b2 # H*V OR d45*d135
    grf = gr1 * gr2
    if grf == 0:
        grf = grf + 0.0001
    cos = gr3 / grf
    if cos > 1:
        cos=1
    if cos < -1:
        cos=-1
    angle = math.acos(cos) * 180.0 / math.pi # arcos (H*V/|H|*|V|) radians to degrees (+0.0001):avoid division by 0
    Nangle = round(angle / 180.0)
    return Nangle

## *GetIntMap* Function

In [ ]:
def GetIntMap(HSV,Qi,Row,Col):
    V=HSV[2,1:Row-1,1:Col-1]
    IM=V*(Qi-1)
    return IM

## *Get_MicroMap* Function

In [ ]:
def Get_MicroMap(S1,S2,Img,Row,Col):
    Ro=Row-2
    Co=Col-2
    A=Get_StructureMap(S1,S2,Img,Ro,Co,0,0)
    B=Get_StructureMap(S1,S2,Img,Ro,Co,0,1)
    C=Get_StructureMap(S1,S2,Img,Ro,Co,1,0)
    D=Get_StructureMap(S1,S2,Img,Ro,Co,1,1)
    Map=np.amax([A,B,C,D],axis=0)
    return Map

### *Get_StructureMap* Function

In [ ]:
def Get_StructureMap(S1,S2,img,Ro,Co,Dx,Dy):
    sMap=np.ones((Ro,Co))*-1
    it = math.floor((Ro-Dx)/3)*(3)#to delet de edge of image and delimit the convolution
    jt = math.floor((Co-Dy)/3)*(3)
    for i in range (1+Dx,it,3):
        for j in range (1+Dy,jt,3):
            WA = S1[i-1:i+2,j-1:j+2]
            WV = S2[i-1:i+2,j-1:j+2]
            for m in range(3):
                for n in range (3):
                    if ((WA[1,1] == WA[m,n]) | (WV[1,1] == WV[m,n])):
                    #if (WA(5) == WA(m,n) && WV(5) == WV(m,n))
                        sMap[i+(m-1), j+(n-1)] = img[i+(m-1), j+(n-1)]
    return sMap

## *Get_SEH* Function

In [ ]:
def Get_SEH(img,Row,Col,te,ne):
    Ro=Row-2
    Co=Col-2
    H1=Get_HiStructures(img,Ro,Co,0,0,te,ne)
    H2=Get_HiStructures(img,Ro,Co,0,1,te,ne)
    H3=Get_HiStructures(img,Ro,Co,1,0,te,ne)
    H4=Get_HiStructures(img,Ro,Co,1,1,te,ne)
    SEH=(H1+H2+H3+H4)/4
    return SEH

### *Get_HiStructures* Function

In [ ]:
def Get_HiStructures(img,Ro,Co,Dx,Dy,te,ne):
    HiStruc=np.zeros(ne)
    it = math.floor((Ro-Dx)/3)*(3)#to delet de edge of image and delimit the convolution
    jt = math.floor((Co-Dy)/3)*(3)
    ct=0
    for i in range (1+Dx,it,3):
        for j in range (1+Dy,jt,3):
            WA = img[i-1:i+2,j-1:j+2]
            [ce,cu,dm,lm,cm] = Get_SCof(WA,0)
            if (te == 1) & (ne == 7) & (ce < 8) & (ce > 0):
                HiStruc[ce-1]=HiStruc[ce-1]+1
            elif (te == 1) & (ne == 9):
                HiStruc[ce]=HiStruc[ce]+1
            elif te == 2:
                T=Get_StypeC(ce,cu,dm,lm,cm,ne)
                if T > -1:
                    HiStruc[T]=HiStruc[T]+1
            ct=ct+1
    HiStruc=HiStruc/ct
    return HiStruc

#### *Get_SCof* Function

In [ ]:
def Get_SCof(WA,WV):
    #Get the coeficients to recognice the esructure element
    if WV == 0:
        WV=WA
    ce=cu=dm=lm=cm=ta=c=0
    vt=np.zeros(4)
    m,n = 0,0
    for p in range (8):
        if (WA[1,1] == WA[m,n]) | (WV[1,1] == WV[m,n]):
        #if WA(5) == WA(m,n) && WV(5) == WV(m,n)
            ce, tp, ta = ce+1, ta, 1
        else:
            tp, ta = ta, 0
        if (p > 0) & (tp != ta):
            if cu == 0:
                ti, ci = tp, c
            cu = cu+1
            if (dm < c) & (tp == 0):
                dm, c = c, 1
            elif (lm < c) & (tp == 1):
                lm, c = c, 1
        else:
            c = c+1
        if p == 7:
            if c < 8:
                if ti == ta:
                    c = c+ci
                else:
                    cu = cu+1
            if (dm < c) & (ta == 0):
                dm = c
            elif (lm < c) & (ta == 1):
                lm = c
        if p <= 3:
            vt[p] = ta
        elif vt[p-4] == ta:
            cm = cm+1
        #Get in circule
        [n,m]=Get_circule(n,p,m)
    return ce,cu,dm,lm,cm

##### *Get_circule* Function

In [ ]:
def Get_circule(n,p,m):
    #get in circule te cordinated
    if (n < 2) & (p < 2):
        n = n+1
    elif m < 2:
        m = m+1
    elif p < 6:
        n = n-1
    else:
        m = m-1
    return n,m

#### *Get_StypeC* Function

In [ ]:
def Get_StypeC(ce,cu,dm,lm,cm,ne):
    if ce == 0:
        t=0
    elif ce == 1:
        t=1

    elif ce == 7:
        t=2

    elif (ce == 2) & (dm == 6):
        t=3
    elif (ce == 2) & (dm == 5):
        t=4
    elif (ce == 2) & (dm == 4):
        t=5
    elif (ce == 2) & (dm == 3):
        t=6

    elif (ce == 6) & (lm == 6):
        t=7
    elif (ce == 6) & (lm == 5):
        t=8
    elif (ce == 6) & (lm == 4):
        t=9
    elif (ce == 6) & (lm == 3):
        t=10

    elif (ce == 3) & (cu == 2):
        t=11
    elif (ce == 3) & (cu == 4) & (dm == 4):
        t=12
    elif (ce == 3) & (cu == 4) & (dm == 3):
        t=13
    elif (ce == 3) & (cu == 6) & (dm == 3):
        t=14
    elif (ce == 3) & (dm == 2):
        t=15

    elif (ce == 5) & (cu == 2):
        t=16
    elif (ce == 5) & (cu == 4) & (lm == 4):
        t=17
    elif (ce == 5) & (cu == 4) & (lm == 3):
        t=18
    elif (ce == 5) & (cu == 6) & (lm == 3):
        t=19
    elif (ce == 5) & (lm == 2):
        t=20

    elif (ce == 4) & (cu == 2):
        t=21
    elif (ce == 4) & (lm == 3) & (dm == 3):
        t=22
    elif (ce == 4) & (lm == 3) & (dm == 2):
        t=23
    elif (ce == 4) & (lm == 2) & (dm == 3):
        t=24
    elif (ce == 4) & (cu == 6) & (cm == 2):
        t=25
    elif (ce == 4) & (cu == 6) & (cm == 0):
        t=26
    elif (ce == 4) & (cu == 4) & (cm == 4):
        t=27
    elif (ce == 4) & (cu == 8):
        t=28
    elif  (ne==30) & (ce == 8):
        t=29
    else:
        t=-1
    if ne == 28:
        t=t-1
    return t

## *Get_MicroMapStr* Function

In [ ]:
def Get_MicroMapStr(S1,S2,Img,Row,Col,te,ne):
    Ro=Row-2
    Co=Col-2
    [A,H1]=Get_StructureMapHS(S1,S2,Img,Ro,Co,0,0,te,ne)
    [B,H2]=Get_StructureMapHS(S1,S2,Img,Ro,Co,0,1,te,ne)
    [C,H3]=Get_StructureMapHS(S1,S2,Img,Ro,Co,1,0,te,ne)
    [D,H4]=Get_StructureMapHS(S1,S2,Img,Ro,Co,1,1,te,ne)
    Map=np.amax([A,B,C,D],axis=0)
    SEH=(H1+H2+H3+H4)/4
    return SEH, Map

### *Get_StructureMapHS* Function

In [ ]:
def Get_StructureMapHS(S1,S2,img,rows,columns,Dx,Dy,te,ne):
    sMap=np.ones((rows, columns))*-1
    HiStruc=np.zeros(ne)
    it = math.floor((rows-Dx)/3)*(3)#to delet de edge of image and delimit the convolution
    jt = math.floor((columns-Dy)/3)*(3)
    ct=0
    for i in range (1+Dx,it,3):
        for j in range (1+Dy,jt,3):
            WA = S1[i-1:i+2,j-1:j+2]
            WV = S2[i-1:i+2,j-1:j+2]
            for m in range(3):
                for n in range (3):
                    if (WA[1,1] == WA[m,n]) | (WV[1,1] == WV[m,n]):
                    #if (WA(5) == WA(m,n) && WV(5) == WV(m,n))
                        sMap[i+(m-1), j+(n-1)] = img[i+(m-1), j+(n-1)]
            [ce,cu,dm,lm,cm] = Get_SCof(WA,0)
            if (te == 1) & (ne == 7) & (ce < 8) & (ce > 0):
                HiStruc[ce-1]=HiStruc[ce-1]+1
            elif (te == 1) & (ne == 9):
                HiStruc[ce]=HiStruc[ce]+1
            elif te == 2:
                T=Get_StypeC(ce,cu,dm,lm,cm,ne)
                if T > -1:
                    HiStruc[T]=HiStruc[T]+1
            ct=ct+1
    HiStruc=HiStruc/ct
    return sMap,HiStruc

## *Get_MSR* Function

In [ ]:
def Get_MSR(Map, CSA, rows,columns):
    MS = np.zeros(CSA)
    HA = np.zeros(CSA)
    hist = np.zeros(CSA)
    for i in range (rows-2):
        for j in range (columns-2):
            Val=int(Map[i,j])
            if Val >= 0:
                HA[Val]= HA[Val]+1
    for i in range (rows-4):
        for j in range (columns-4):
            wa = Map[i:i+3,j:j+3]
            TE1 = 0
            for m in range (3):
                for n in range (3):
                    if ((wa[1,1] == wa[m,n]) & (wa[1,1] >= 0)):
                        if (m != 1) & (n!= 1):
                            TE1 = TE1 + 1
            Val=int(wa[1,1])
            if Val >= 0:
                MS[Val] = MS[Val]+TE1
    for i in range (CSA):
        if HA[i] > 0.0:
            hist[i] = (MS[i] * 1.0) / (8.0 * HA[i])
    return hist